In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split, Subset
import time
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split

import pickle
from tqdm import tqdm
import copy

In [2]:
import pyro
import pyro.distributions as dist
from pyro.nn import PyroModule, PyroSample
from pyro.infer.autoguide import AutoDiagonalNormal

c:\Users\Revalda Putawara\.conda\envs\bnntest\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import pandas as pd

In [4]:
import numpy as np
from sklearn.metrics import confusion_matrix

In [5]:
from bitflip import bitflip_float32

In [6]:
from torchvision.datasets import ImageFolder

In [7]:
shipsnet_mean = [0.4119, 0.4243, 0.3724]
shipsnet_std = [0.1899, 0.1569, 0.1515]

def load_data(batch_size=16):
    transform = transforms.Compose([
        transforms.Resize((64, 64)),
        transforms.ToTensor(),
        transforms.Normalize(mean=shipsnet_mean, 
                             std=shipsnet_std)
    ])

    #dataset = datasets.EuroSAT(root='./data', transform=transform, download=True)
    dataset = ImageFolder(
    root="data/shipsnet/foldered",
    transform=transform
    )
    torch.manual_seed(42)

    #train_size = int(0.8 * len(dataset))
    #test_size = len(dataset) - train_size
    #train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
    
    with open('datasplit/shipsnet_split_indices.pkl', 'rb') as f:
        split = pickle.load(f)
        train_dataset = Subset(dataset, split['train'])
        test_dataset = Subset(dataset, split['test'])

    # Add num_workers and pin_memory for faster data loading
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, 
                             num_workers=4, pin_memory=True, persistent_workers=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size,
                            num_workers=4, pin_memory=True, persistent_workers=True)
    return train_loader, test_loader

In [8]:
train_loader, test_loader = load_data(batch_size=16)

In [9]:
device = torch.device("cuda")

In [10]:
num_classes = 2

In [25]:
class BayesShipsCNN(PyroModule):
    def __init__(
        self,
        num_classes=2,
        device= torch.device("cuda"),
        activation='relu',
        prior_dist='gaussian',
        mu=0.0,
        b=1.0,
        prior_params=None
    ):
        super().__init__()

        # Store device
        self.device = device

        # Activation setup: accept string or callable
        if isinstance(activation, str):
            act_map = {
                'relu': F.relu,
                'tanh': F.tanh,
                'sigmoid': F.sigmoid,
                'sinusoidal': torch.sin,
                'relu6': F.relu6,
                'leaky_relu': F.leaky_relu,
                'selu': F.selu,
                'wg':self.actWG,
                'rwg':self.actRWG,
            }
            try:
                self.activation_fn = act_map[activation]
            except KeyError:
                raise ValueError(f"Unsupported activation: {activation}")
        elif callable(activation):
            self.activation_fn = activation
        else:
            raise ValueError("activation must be a string or callable")

        # Prior distribution setup
        self.prior_dist = prior_dist
        default_params = {'mu': mu, 'b': b}
        params = default_params if prior_params is None else prior_params
        self.prior_mu = torch.tensor(params.get('mu', mu), device=device)
        self.prior_b  = torch.tensor(params.get('b', b), device=device)

        print(f"Using activation function: {activation}")
        print(f"Using prior distribution: {self.prior_dist} with mu={self.prior_mu.item()} and b={self.prior_b.item()}")

        # Layer definitions with priors
        self.conv1 = PyroModule[nn.Conv2d](3, 32, kernel_size=3, stride=1, padding=1)
        self.conv1.weight = PyroSample(self._make_prior([32, 3, 3, 3]))
        self.conv1.bias   = PyroSample(self._make_prior([32]))

        self.conv2 = PyroModule[nn.Conv2d](32, 64, kernel_size=3, stride=1, padding=1)
        self.conv2.weight = PyroSample(self._make_prior([64, 32, 3, 3]))
        self.conv2.bias   = PyroSample(self._make_prior([64]))

        self.conv3 = PyroModule[nn.Conv2d](64, 128, kernel_size=3, stride=1, padding=1)
        self.conv3.weight = PyroSample(self._make_prior([128, 64, 3, 3]))
        self.conv3.bias   = PyroSample(self._make_prior([128]))

        # Pooling and global average pooling
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.gap  = nn.AdaptiveAvgPool2d((1, 1))

        # Fully connected layers
        self.fc1 = PyroModule[nn.Linear](128, 256)
        self.fc1.weight = PyroSample(self._make_prior([256, 128]))
        self.fc1.bias   = PyroSample(self._make_prior([256]))

        self.fc2 = PyroModule[nn.Linear](256, num_classes)
        self.fc2.weight = PyroSample(self._make_prior([num_classes, 256]))
        self.fc2.bias   = PyroSample(self._make_prior([num_classes]))

    def actWG(self, x, alpha=1.0):
        # Weight-gradient activation
        return x * torch.exp(-alpha * x ** 2)
    
    def actRWG(self, x, alpha=1.0):
        wg = x * torch.exp(-alpha * x ** 2)
        # compare elementwise with zero
        return torch.max(torch.zeros_like(wg), wg)

    def _make_prior(self, shape):
        """
        Construct a prior distribution based on self.prior_dist and parameters.
        """
        if self.prior_dist == 'gaussian':
            base = dist.Normal(self.prior_mu, self.prior_b)
        elif self.prior_dist == 'laplace':
            base = dist.Laplace(self.prior_mu, self.prior_b)
        elif self.prior_dist == 'uniform':
            base = dist.Uniform(-self.prior_b, self.prior_b)
        else:
            raise ValueError(f"Unsupported prior distribution: {self.prior_dist}")
        return base.expand(shape).to_event(len(shape))

    def forward(self, x, y=None):
        x = self.activation_fn(self.conv1(x).to(self.device))
        x = self.pool(x)

        x = self.activation_fn(self.conv2(x))
        x = self.pool(x)

        x = self.activation_fn(self.conv3(x))
        x = self.pool(x)

        x = self.gap(x)
        x = x.view(x.size(0), -1)

        x = self.activation_fn(self.fc1(x))
        logits = self.fc2(x)

        if y is not None:
            with pyro.plate("data", x.size(0)):
                pyro.sample("obs", dist.Categorical(logits=logits), obs=y)
        return logits

In [50]:
model_param_index = {
    'conv1': {
        'weight': {
            'shape': (32, 3, 3, 3),
            'start_idx': 0,
            'end_idx': 863,
            'total_size': 864
        },
        'bias': {
            'shape': (32,),
            'start_idx': 864,
            'end_idx': 895,
            'total_size': 32
        }
    },
    'conv2': {
        'weight': {
            'shape': (64, 32, 3, 3),
            'start_idx': 896,
            'end_idx': 19327,
            'total_size': 18432
        },
        'bias': {
            'shape': (64,),
            'start_idx': 19328,
            'end_idx': 19391,
            'total_size': 64
        }
    },
    'conv3': {
        'weight': {
            'shape': (128, 64, 3, 3),
            'start_idx': 19392,
            'end_idx': 93119,
            'total_size': 73728
        },
        'bias': {
            'shape': (128,),
            'start_idx': 93120,
            'end_idx': 93247,
            'total_size': 128
        }
    },
    'fc1': {
        'weight': {
            'shape': (256, 128),
            'start_idx': 93248,
            'end_idx': 126015,
            'total_size': 32768
        },
        'bias': {
            'shape': (256,),
            'start_idx': 126016,
            'end_idx': 126271,
            'total_size': 256
        }
    },
    'fc2': {
        'weight': {
            'shape': (2, 256),
            'start_idx': 126272,
            'end_idx': 126783,
            'total_size': 512
        },
        'bias': {
            'shape': (2,),
            'start_idx': 126784,
            'end_idx': 126785,
            'total_size': 2
        }
    }
}

In [26]:
import os

search_dir = 'inj_base_ships'
#list all .json files in the directory
all_files = [f for f in os.listdir(search_dir)]
json_files = [f for f in os.listdir(search_dir) if f.endswith('.json')]

# excluding the format, get the last 16 characters of each filename
timestamps = [f[:-5][-16:] for f in json_files]

# for each timestamp, look for every other files in the directory that contains the timestamp

config_files = {}
guide_files = {}
model_files = {}
param_files = {}

for timestamp in timestamps:
    config_files[timestamp] = [f for f in all_files if timestamp in f and f.endswith('.json')][0]
    guide_files[timestamp] = [f for f in all_files if timestamp in f and f.startswith('guide')][0]
    model_files[timestamp] = [f for f in all_files if timestamp in f and f.startswith('model')][0]
    param_files[timestamp] = [f for f in all_files if timestamp in f and f.startswith('param')][0]


In [27]:
config_files

{'_20250715_165043': 'config_relu_gaussian_20250715_165043.json',
 '_20250715_160541': 'config_tanh_gaussian_20250715_160541.json'}

In [28]:
#load the model
import json

def load_model(timestamp):
    config_path = os.path.join(search_dir, config_files[timestamp])
    guide_path = os.path.join(search_dir, guide_files[timestamp])
    model_path = os.path.join(search_dir, model_files[timestamp])
    param_path = os.path.join(search_dir, param_files[timestamp])

    print(f"Loading model with config_path: {config_path}")

    with open(config_path, 'r') as f:
        config = json.load(f)

    model = BayesShipsCNN(
        num_classes=num_classes,
        device=device,
        activation=config['activation'],
        prior_dist=config['prior'],
        mu=config['prior_params']['mu'],
        b=config['prior_params'],
        prior_params=config.get('prior_params', None)
    ).to(device)

    # Load the guide
    #guide = AutoDiagonalNormal(model)

    # Load the model state
    model.load_state_dict(torch.load(model_path))
    
    # Load the guide state
    #guide.load_state_dict(torch.load(guide_path))

    return model, param_path

In [29]:
timestamps

['_20250715_165043', '_20250715_160541']

In [30]:
bayesian_model, pyro_param_store_path = load_model(timestamps[1])

Loading model with config_path: inj_base_ships\config_tanh_gaussian_20250715_160541.json
Using activation function: tanh
Using prior distribution: gaussian with mu=0.0 and b=10.0


In [ ]:
#prior_iter = 'gaussian'
#activation_iter = 'relu'
#b_iter = 10.0

#bayesian_model = BayesShipsCNN(num_classes,
#                        device,
#                        activation=activation_iter,
#                        prior_dist=prior_iter,
#                        mu = 0.0,
#                        b= b_iter,
#                        #prior_params={'mu': 0.0, 'b': b_iter})
#                        )

In [ ]:
#model_path = 'inj_base_ships/model_relu_gaussian_epoch_best_20250715_165043.pth'
#guide_path = 'inj_base_ships/guide_relu_gaussian_epoch_best_20250715_165043.pth'
#pyro_param_store_path = 'inj_base_ships/param_store_relu_gaussian_epoch_best_20250715_165043.pkl'

#guide = AutoDiagonalNormal(bayesian_model).to(device)

#pyro.get_param_store().set_state(torch.load(pyro_param_store_path,weights_only=False))

#original_param_store = {}

#for name, value in pyro.get_param_store().items():
#    print(f"{name}: {value.shape}")
#    original_param_store[name] = torch.tensor(value.data, requires_grad=value.requires_grad)

In [ ]:
class Injector:
    def __init__(self, trained_model, device, test_loader, num_samples):
        """
        Initializes SEU injector
        """

        #initialize device
        if device is None:
            self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        else:
            self.device = device

        self.trained_model = trained_model.to(self.device)
        self.test_loader = test_loader
        self.trained_model.eval()
        self.num_samples = num_samples
        
        self.guide = AutoDiagonalNormal(self.trained_model).to(self.device)
        pyro.get_param_store().clear()
        pyro.get_param_store().set_state(torch.load(pyro_param_store_path,weights_only=False))

        #initial_Accuracy = self.return_accuracy(num_samples)
        initial_labels, initial_predictions, initial_logits, initial_probs = self.predict_data_probs(self.num_samples)
        self.initial_accuracy = self.return_accuracy(initial_labels, initial_predictions)
        
        #print accuracy before SEU
        print(f"Initial accuracy: {self.initial_accuracy}")
        
    def predict_data_probs(self, num_samples=10):
        all_labels = []
        all_predictions = []
        all_logits = []
        all_probs = []

        with torch.no_grad():
            for images, labels in tqdm(self.test_loader, desc="Evaluating"):
                images, labels = images.to(self.device), labels.to(self.device)

                logits_mc = torch.zeros(num_samples, images.size(0), self.trained_model.fc2.out_features).to(device)

                for i in range(num_samples):
                    guide_trace = pyro.poutine.trace(self.guide).get_trace(images)
                    replayed_model = pyro.poutine.replay(self.trained_model, trace=guide_trace)
                    logits = replayed_model(images)
                    logits_mc[i] = logits

                avg_logits = logits_mc.mean(dim=0)
                predictions = torch.argmax(avg_logits, dim=1)

                all_labels.extend(labels.cpu().numpy())
                all_predictions.extend(predictions.cpu().numpy())
                all_logits.extend(avg_logits.cpu().numpy())
                all_probs.extend(F.softmax(avg_logits, dim=1).cpu().numpy())

        return all_labels, all_predictions, all_logits, all_probs

    def return_accuracy(self, all_labels, all_predictions):
        #all_labels, all_predictions, all_logits, all_probs = self.predict_data_probs(num_samples)
        cm = confusion_matrix(all_labels, all_predictions)
        accuracy = np.trace(cm) / np.sum(cm)

        return accuracy
    
    def run_seu_autodiagonal_normal(self, location_index, bit_i, parameter_name="loc", num_samples=10):
        """
        Run SEU on the AutoDiagonalNormal guide
        """

        assert parameter_name in ["loc", "scale"], "Parameter name must be 'loc' or 'scale'."
        assert bit_i in range(0, 33), "Location index must be between 0 and 9."

        if parameter_name == "loc":
            param_store_name = "AutoDiagonalNormal.loc"
        elif parameter_name == "scale":
            param_store_name = "AutoDiagonalNormal.scale"

        pyro.get_param_store().set_state(torch.load(pyro_param_store_path,weights_only=False))

        with torch.no_grad():
            param_dict = {}

            for name, value in pyro.get_param_store().items():
                #print(f"{name}: {value.shape}")
                #print(value)
                param_dict[name] = value.cpu().detach().numpy()

            tensor_cpu = param_dict[param_store_name]

            #original_val = tensor_cpu[0] #this zero index should be changed to the location_index
            original_val = tensor_cpu[location_index]
            seu_val = bitflip_float32(original_val, bit_i)


            print(f"Original value: {original_val}, SEU value: {seu_val}")

            # Get the parameter
            param = pyro.get_param_store().get_param(param_store_name)

            # Modify it safely by creating a new tensor
            new_param = param.clone()
            new_param[location_index] = seu_val  # New Value

            # Update the parameter store
            if parameter_name == "loc":
                pyro.get_param_store().__setitem__(param_store_name, new_param) # 74%
                #param_store[param_store_name].data.copy_(change_item(param_store_name, location_index, seu_val)) #25%
                #pyro.get_param_store()[param_store_name].data[location_index] = seu_val # 25%
            elif parameter_name == "scale":
                pyro.get_param_store().__setitem__(param_store_name, new_param) #10%
                #pyro.get_param_store()[param_store_name].data[location_index] = seu_val

        #print accuracy after SEU
        self.guide = AutoDiagonalNormal(bayesian_model).to(device)
    

        try:
            after_labels, after_predictions, after_logits, after_probs = self.predict_data_probs(num_samples)
            accuracyAfter_SEU = self.return_accuracy(after_labels, after_predictions)
        except:
            accuracyAfter_SEU = np.nan
            
        print(f"Accuracy after SEU: {accuracyAfter_SEU}")
        print("===================================")

        return accuracyAfter_SEU - self.initial_accuracy
    
    def run_seu_autodiagonal_normal_multi(self, location_indices, bit_i, parameter_name="loc", attack_ratio=1.0, num_samples=10, seed=None):
        """
        Run SEU on the AutoDiagonalNormal guide with multiple location attacks
        
        Args:
            location_indices: list of indices or single index to attack
            bit_i: bit position to flip (0-32)
            parameter_name: "loc" or "scale"
            attack_ratio: ratio of locations to attack (0.0-1.0)
            num_samples: number of MC samples for evaluation
            seed: random seed for reproducible attacks
        """
        
        assert parameter_name in ["loc", "scale"], "Parameter name must be 'loc' or 'scale'."
        assert bit_i in range(0, 33), "Bit index must be between 0 and 32."
        assert 0.0 <= attack_ratio <= 1.0, "Attack ratio must be between 0.0 and 1.0."
        
        # Convert single index to list for uniform handling
        if isinstance(location_indices, int):
            location_indices = [location_indices]
        
        # Set random seed if provided
        if seed is not None:
            np.random.seed(seed)
            torch.manual_seed(seed)
        
        # Calculate number of locations to attack
        num_attacks = max(1, int(len(location_indices) * attack_ratio))
        
        # Randomly select locations to attack
        attack_locations = np.random.choice(location_indices, size=num_attacks, replace=False)
        
        if parameter_name == "loc":
            param_store_name = "AutoDiagonalNormal.loc"
        elif parameter_name == "scale":
            param_store_name = "AutoDiagonalNormal.scale"

        # Reset parameter store to original state
        pyro.get_param_store().set_state(torch.load(pyro_param_store_path, weights_only=False))

        with torch.no_grad():
            # Get the parameter
            param = pyro.get_param_store().get_param(param_store_name)
            
            # Create a new tensor to modify
            new_param = param.clone()
            
            print(f"Attacking {num_attacks} out of {len(location_indices)} locations:")
            
            # Attack each selected location
            for location_index in attack_locations:
                original_val = new_param[location_index].cpu().item()
                seu_val = bitflip_float32(original_val, bit_i)
                new_param[location_index] = seu_val
                
                print(f"  Location {location_index}: {original_val} -> {seu_val}")
            
            # Update the parameter store
            pyro.get_param_store().__setitem__(param_store_name, new_param)

        # Reinitialize guide with modified parameters
        self.guide = AutoDiagonalNormal(bayesian_model).to(device)

        try:
            after_labels, after_predictions, after_logits, after_probs = self.predict_data_probs(num_samples)
            accuracyAfter_SEU = self.return_accuracy(after_labels, after_predictions)
        except:
            accuracyAfter_SEU = np.nan
            
        print(f"Accuracy after SEU: {accuracyAfter_SEU}")
        print("===================================")

        return accuracyAfter_SEU - self.initial_accuracy
    
#inj = Injector(trained_model=bayesian_model, device=device, test_loader=test_loader, num_samples=10)

In [43]:
import torch
import torch.nn.functional as F
import numpy as np
import pyro
from pyro.infer.autoguide import AutoDiagonalNormal
from sklearn.metrics import confusion_matrix
from tqdm import tqdm
import math

class NewInjector:
    def __init__(self, trained_model, device, test_loader, num_samples):
        """
        Initializes SEU injector
        """
        self.device = device or torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.trained_model = trained_model.to(self.device)
        self.test_loader = test_loader
        self.trained_model.eval()
        self.num_samples = num_samples

        self.guide = AutoDiagonalNormal(self.trained_model).to(self.device)
        pyro.get_param_store().clear()
        pyro.get_param_store().set_state(torch.load(pyro_param_store_path, weights_only=False))

        initial_labels, initial_predictions, initial_logits, initial_probs = self.predict_data_probs(self.num_samples)
        self.initial_accuracy = self.return_accuracy(initial_labels, initial_predictions)
        self.initial_probs = np.array(initial_probs)

        print(f"Initial accuracy: {self.initial_accuracy}")

    def predict_data_probs(self, num_samples=10):
        all_labels = []
        all_predictions = []
        all_logits = []
        all_probs = []

        with torch.no_grad():
            for images, labels in tqdm(self.test_loader, desc="Evaluating"):
                images, labels = images.to(self.device), labels.to(self.device)
                logits_mc = torch.zeros(num_samples, images.size(0), self.trained_model.fc2.out_features).to(self.device)

                for i in range(num_samples):
                    guide_trace = pyro.poutine.trace(self.guide).get_trace(images)
                    replayed_model = pyro.poutine.replay(self.trained_model, trace=guide_trace)
                    logits = replayed_model(images)
                    logits_mc[i] = logits

                avg_logits = logits_mc.mean(dim=0)
                predictions = torch.argmax(avg_logits, dim=1)

                all_labels.extend(labels.cpu().numpy())
                all_predictions.extend(predictions.cpu().numpy())
                all_logits.extend(avg_logits.cpu().numpy())
                all_probs.extend(F.softmax(avg_logits, dim=1).cpu().numpy())

        return all_labels, all_predictions, all_logits, all_probs

    def return_accuracy(self, all_labels, all_predictions):
        cm = confusion_matrix(all_labels, all_predictions)
        return np.trace(cm) / np.sum(cm)

    def compute_softmax_difference(self, before_probs, after_probs):
        before_probs = np.array(before_probs)
        after_probs = np.array(after_probs)
        diff = np.abs(before_probs - after_probs)
        return np.max(diff, axis=1).mean()

    def compute_difference(self, original_val, modified_val):
        return abs(original_val - modified_val)

    def run_seu_autodiagonal_normal(self, location_index, bit_i, parameter_name="loc", num_samples=10):
        assert parameter_name in ["loc", "scale"], "Parameter name must be 'loc' or 'scale'."
        assert bit_i in range(0, 33), "Bit index must be between 0 and 32."

        param_store_name = f"AutoDiagonalNormal.{parameter_name}"
        pyro.get_param_store().set_state(torch.load(pyro_param_store_path, weights_only=False))

        with torch.no_grad():
            param = pyro.get_param_store().get_param(param_store_name)
            new_param = param.clone()
            original_val = new_param[location_index].cpu().item()
            seu_val = bitflip_float32(original_val, bit_i)
            abs_diff = self.compute_difference(original_val, seu_val)
            new_param[location_index] = seu_val
            pyro.get_param_store().__setitem__(param_store_name, new_param)

            print(f"Original value: {original_val}, SEU value: {seu_val}, Abs difference: {abs_diff}")

        self.guide = AutoDiagonalNormal(self.trained_model).to(self.device)

        try:
            after_labels, after_predictions, after_logits, after_probs = self.predict_data_probs(num_samples)
            accuracy_after = self.return_accuracy(after_labels, after_predictions)
            softmax_diff = self.compute_softmax_difference(self.initial_probs, after_probs)
        except:
            accuracy_after = np.nan
            softmax_diff = np.nan

        print(f"Accuracy after SEU: {accuracy_after}")
        print("===================================")

        return {
            "accuracy_change": accuracy_after - self.initial_accuracy,
            "softmax_difference": softmax_diff,
            "absolute_difference": abs_diff
        }

    def run_seu_autodiagonal_normal_multi(self, location_indices, bit_i, parameter_name="loc",
                                          attack_ratio=1.0, num_samples=10, seed=None):
        assert parameter_name in ["loc", "scale"], "Parameter name must be 'loc' or 'scale'."
        assert bit_i in range(0, 33), "Bit index must be between 0 and 32."
        assert 0.0 <= attack_ratio <= 1.0, "Attack ratio must be between 0.0 and 1.0."

        if isinstance(location_indices, int):
            location_indices = [location_indices]

        if seed is not None:
            np.random.seed(seed)
            torch.manual_seed(seed)

        num_attacks = max(1, int(len(location_indices) * attack_ratio))
        attack_locations = np.random.choice(location_indices, size=num_attacks, replace=False)
        param_store_name = f"AutoDiagonalNormal.{parameter_name}"
        pyro.get_param_store().set_state(torch.load(pyro_param_store_path, weights_only=False))

        abs_differences = []

        with torch.no_grad():
            param = pyro.get_param_store().get_param(param_store_name)
            new_param = param.clone()

            #print(f"Attacking {num_attacks} out of {len(location_indices)} locations:")

            for location_index in attack_locations:
                original_val = new_param[location_index].cpu().item()
                seu_val = bitflip_float32(original_val, bit_i)
                abs_diff = self.compute_difference(original_val, seu_val)
                abs_differences.append(abs_diff)
                new_param[location_index] = seu_val
                print(f"  Location {location_index}: {original_val} -> {seu_val}, Log diff: {abs_diff}")

            pyro.get_param_store().__setitem__(param_store_name, new_param)

        self.guide = AutoDiagonalNormal(self.trained_model).to(self.device)

        try:
            after_labels, after_predictions, after_logits, after_probs = self.predict_data_probs(num_samples)
            accuracy_after = self.return_accuracy(after_labels, after_predictions)
            softmax_diff = self.compute_softmax_difference(self.initial_probs, after_probs)
            mean_abs_diff = float(np.mean(abs_differences))
        except:
            accuracy_after = np.nan
            softmax_diff = np.nan
            mean_abs_diff = np.nan

        #print(f"Accuracy after SEU: {accuracy_after}")
        #print("===================================")

        return {
            "accuracy_change": accuracy_after - self.initial_accuracy,
            "softmax_difference": softmax_diff,
            "mean_abs_difference": mean_abs_diff
        }


In [44]:
newinj = NewInjector(trained_model=bayesian_model, device=device, test_loader=test_loader, num_samples=10)

Evaluating: 100%|██████████| 50/50 [00:03<00:00, 13.71it/s]

Initial accuracy: 0.71625


In [45]:
def load_model_config(timestamp):
    config_path = os.path.join(search_dir, config_files[timestamp])

    with open(config_path, 'r') as f:
        model_config = json.load(f)

    return model_config

In [49]:
# print pyro param store name and shape
for name, value in pyro.get_param_store().items():
    print(f"{name}: {value.shape}")

AutoDiagonalNormal.loc: torch.Size([126786])
AutoDiagonalNormal.scale: torch.Size([126786])


In [46]:
timestamps

['_20250715_165043', '_20250715_160541']

In [47]:
model_config = load_model_config(timestamps[1])
model_config

{'activation': 'tanh',
 'prior': 'gaussian',
 'num_epochs': 100,
 'best_accuracy_at_epoch': 10,
 'best_accuracy': 0.7140625,
 'batch_size': 16,
 'train_size': 3200,
 'prior_params': {'mu': 0.0, 'b': 10.0}}

In [ ]:
# loop through model param index


In [52]:
# loop through model param index
for layer, params in model_param_index.items():
    for param_name, param_info in params.items():
        print(f"{layer}.{param_name}: {param_info['shape']} (start: {param_info['start_idx']}, end: {param_info['end_idx']})")

conv1.weight: (32, 3, 3, 3) (start: 0, end: 863)
conv1.bias: (32,) (start: 864, end: 895)
conv2.weight: (64, 32, 3, 3) (start: 896, end: 19327)
conv2.bias: (64,) (start: 19328, end: 19391)
conv3.weight: (128, 64, 3, 3) (start: 19392, end: 93119)
conv3.bias: (128,) (start: 93120, end: 93247)
fc1.weight: (256, 128) (start: 93248, end: 126015)
fc1.bias: (256,) (start: 126016, end: 126271)
fc2.weight: (2, 256) (start: 126272, end: 126783)
fc2.bias: (2,) (start: 126784, end: 126785)


In [ ]:
attack_locations = ["beginning", "end"]

for attack_location in attack_locations:
    for layer, params in model_param_index.items():
        for param_name, param_info in params.items():
            
            if attack_location == "beginning":
                target_index = param_info['start_idx']
            elif attack_location == "end":
                target_index = param_info['end_idx']

            print(f"Running SEU on {layer}.{param_name} at index {target_index} with bit flip 0")

Running SEU on conv1.weight at index 0 with bit flip 0
Running SEU on conv1.bias at index 864 with bit flip 0
Running SEU on conv2.weight at index 896 with bit flip 0
Running SEU on conv2.bias at index 19328 with bit flip 0
Running SEU on conv3.weight at index 19392 with bit flip 0
Running SEU on conv3.bias at index 93120 with bit flip 0
Running SEU on fc1.weight at index 93248 with bit flip 0
Running SEU on fc1.bias at index 126016 with bit flip 0
Running SEU on fc2.weight at index 126272 with bit flip 0
Running SEU on fc2.bias at index 126784 with bit flip 0
Running SEU on conv1.weight at index 863 with bit flip 0
Running SEU on conv1.bias at index 895 with bit flip 0
Running SEU on conv2.weight at index 19327 with bit flip 0
Running SEU on conv2.bias at index 19391 with bit flip 0
Running SEU on conv3.weight at index 93119 with bit flip 0
Running SEU on conv3.bias at index 93247 with bit flip 0
Running SEU on fc1.weight at index 126015 with bit flip 0
Running SEU on fc1.bias at inde

In [57]:
## MAIN LOOP CODE

results_df = pd.DataFrame(columns=["activation_fn",
                                   "prior",
                                   "best_accuracy",
                                   "prior_mu",
                                    "prior_b",
                                    "param_type",
                                   "location_index",
                                   "location_layer",
                                   "location_module",
                                   "bit_index", 
                                   "initial_accuracy", 
                                   "accuracy_after_seu", 
                                   "accuracy_change", 
                                   "softmax_difference", 
                                   "mean_abs_difference",
                                   ])

# get the initial accuracy from the newinj object
initial_accuracy = newinj.initial_accuracy

attack_locations = ["beginning", "end"]

for attack_location in attack_locations:
    for layer, params in model_param_index.items():
        for param_name, param_info in params.items():
            
            if attack_location == "beginning":
                target_index = param_info['start_idx']
            elif attack_location == "end":
                target_index = param_info['end_idx']

            #print(f"Running SEU on {layer}.{param_name} at index {target_index} with bit flip 0")

            for bit_iter in range(0, 3):
                for parameter_name in ["loc", "scale"]:
                    print(f"Running SEU for bit index {bit_iter} for parameter {parameter_name}")
                    result = newinj.run_seu_autodiagonal_normal(location_index=target_index, bit_i=bit_iter, parameter_name=parameter_name, num_samples=10)
                    # use concat to save the result to a dataframe
                    #print(initial_accuracy)
                    iter_df = pd.DataFrame({
                        "activation_fn": model_config['activation'],
                        "prior": model_config['prior'],
                        "best_accuracy": model_config['best_accuracy'],
                        "prior_mu": model_config['prior_params']['mu'],
                        "prior_b": model_config['prior_params']['b'],
                        "param_type": parameter_name,
                        "location_index": target_index,
                        "location_layer": layer,
                        "location_module": param_name,
                        "bit_index": bit_iter,
                        "initial_accuracy": initial_accuracy,
                        "accuracy_after_seu": initial_accuracy + result["accuracy_change"],
                        "accuracy_change": result["accuracy_change"],
                        "softmax_difference": result["softmax_difference"],
                        "mean_abs_difference": result["absolute_difference"]
                    }, index=[0])

                    results_df = pd.concat([results_df, iter_df], ignore_index=True)

results_df

Running SEU for bit index 0 for parameter loc
Original value: 2.0283055305480957, SEU value: -2.0283055305480957, Abs difference: 4.056611061096191


Evaluating: 100%|██████████| 50/50 [00:04<00:00, 11.37it/s]
C:\Users\Revalda Putawara\AppData\Local\Temp\ipykernel_32748\1048480182.py:60: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, iter_df], ignore_index=True)


Accuracy after SEU: 0.735
Running SEU for bit index 0 for parameter scale
Original value: 0.10241174697875977, SEU value: -0.10241174697875977, Abs difference: 0.20482349395751953


Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]


Accuracy after SEU: nan
Running SEU for bit index 1 for parameter loc
Original value: 2.0283055305480957, SEU value: 1.6636495628157093e-40, Abs difference: 2.0283055305480957


...

Evaluating: 100%|██████████| 50/50 [00:03<00:00, 13.20it/s]


Accuracy after SEU: 0.25
Running SEU for bit index 2 for parameter scale
Original value: 0.46620631217956543, SEU value: 2.5273094824577006e-20, Abs difference: 0.46620631217956543


Evaluating: 100%|██████████| 50/50 [00:03<00:00, 12.99it/s]

Accuracy after SEU: 0.74


,activation_fn,prior,best_accuracy,prior_mu,prior_b,param_type,location_index,location_layer,location_module,bit_index,initial_accuracy,accuracy_after_seu,accuracy_change,softmax_difference,mean_abs_difference
0,tanh,gaussian,0.714063,0.0,10.0,loc,0,conv1,weight,0,0.71625,0.73500,0.01875,0.084771,4.056611e+00
1,tanh,gaussian,0.714063,0.0,10.0,scale,0,conv1,weight,0,0.71625,NaN,NaN,NaN,2.048235e-01
2,tanh,gaussian,0.714063,0.0,10.0,loc,0,conv1,weight,1,0.71625,0.73000,0.01375,0.084719,2.028306e+00
3,tanh,gaussian,0.714063,0.0,10.0,scale,0,conv1,weight,1,0.71625,0.72625,0.01000,0.191886,3.484891e+37
4,tanh,gaussian,0.714063,0.0,10.0,loc,0,conv1,weight,2,0.71625,0.73250,0.01625,0.109688,3.741563e+19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,tanh,gaussian,0.714063,0.0,10.0,scale,126785,fc2,bias,0,0.71625,NaN,NaN,NaN,9.324126e-01
116,tanh,gaussian,0.714063,0.0,10.0,loc,126785,fc2,bias,1,0.71625,0.75000,0.03375,0.085478,1.636517e+01
117,tanh,gaussian,0.714063,0.0,10.0,scale,126785,fc2,bias,1,0.71625,0.50250,-0.21375,NaN,1.586418e+38
118,tanh,gaussian,0.714063,0.0,10.0,loc,126785,fc2,bias,2,0.71625,0.25000,-0.46625,0.771068,3.018841e+20


In [58]:
# save the results to a csv file
results_df.to_csv(f'shipsnet_seu_result/{timestamps[1]}.csv', index=False)

In [ ]:
#inj.run_seu_autodiagonal_normal(location_index=0, bit_i=1, parameter_name="scale", num_samples=10)
#inj.run_seu_autodiagonal_normal(location_index=0, bit_i=2, parameter_name="loc", num_samples=10)

In [ ]:
#store the result in dataframe
import pandas as pd
results = []

for i in range(0, 32):
    results.append(inj.run_seu_autodiagonal_normal(location_index=0, bit_i=i, parameter_name="scale", num_samples=10))

In [ ]:
# plot the result in matplotlib
plt.figure(figsize=(10, 5))
plt.plot(range(0, 32), results, marker='o')
plt.title('SEU Impact on Scale Parameter')
plt.xlabel('Bit Position')
plt.ylabel('Change in Accuracy')
plt.grid()
plt.show()

In [ ]:
results_loc = []

for i in range(0, 32):
    results_loc.append(inj.run_seu_autodiagonal_normal(location_index=0, bit_i=i, parameter_name="loc", num_samples=10))

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(range(0, 32), results_loc, marker='o')
plt.title('SEU Impact on Loc Parameter')
plt.xlabel('Bit Position')
plt.ylabel('Change in Accuracy')
plt.grid()
plt.show()

In [ ]:
#(inj.run_seu_autodiagonal_normal(location_index=0, bit_i=0, parameter_name="loc", num_samples=10))

In [ ]:
results2_loc = []

for i in range(0, 32):
    results2_loc.append(inj.run_seu_autodiagonal_normal(location_index=217536, bit_i=i, parameter_name="loc", num_samples=10))

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(range(0, 32), results2_loc, marker='o')
plt.title('SEU Impact on Loc Parameter')
plt.xlabel('Bit Position')
plt.ylabel('Change in Accuracy')
plt.grid()
plt.show()

In [ ]:
bayesian_model

In [ ]:
# print the parameters of the bayesian model
for name, value in bayesian_model.named_parameters():
    print(f"{name}: {value.shape}")

In [ ]:
# print the parameters of pyro param store
for name, value in pyro.get_param_store().items():
    print(f"{name}: {value.shape}")

In [ ]:
results3_loc = []

for i in range(0, 32):
    results3_loc.append(inj.run_seu_autodiagonal_normal(location_index=53696, bit_i=i, parameter_name="loc", num_samples=10))

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(range(0, 32), results3_loc, marker='o')
plt.title('SEU Impact on Loc Parameter')
plt.xlabel('Bit Position')
plt.ylabel('Change in Accuracy')
plt.grid()
plt.show()

In [ ]:
import numpy as np

def get_flat_param_index(layer_name, param_type, index_tuple):
    # Define parameter shapes for each layer
    param_shapes = {
        'conv1': {
            'weight': (32, 3, 5, 5),
            'bias': (32,)
        },
        'conv2': {
            'weight': (64, 32, 5, 5),
            'bias': (64,)
        },
        'fc1': {
            'weight': (10, 16384),
            'bias': (10,)
        }
    }

    # Check valid inputs
    if layer_name not in param_shapes:
        raise ValueError(f"Unknown layer '{layer_name}'")
    if param_type not in param_shapes[layer_name]:
        raise ValueError(f"'{param_type}' not found in layer '{layer_name}'")

    # Compute flat offset for each param in order: conv1.weight, conv1.bias, ...
    flat_offset = 0
    for l in ['conv1', 'conv2', 'fc1']:
        for p in ['weight', 'bias']:
            shape = param_shapes[l][p]
            numel = np.prod(shape)

            if l == layer_name and p == param_type:
                # Compute local flat index within this param
                local_flat_index = np.ravel_multi_index(index_tuple, shape)
                return flat_offset + local_flat_index

            flat_offset += numel

    raise RuntimeError("Should not reach here if input is valid.")


In [ ]:
# Index of conv1.bias[5]
print(get_flat_param_index("conv1", "bias", (0,)))  # Output: 2405

# Index of conv2.weight[0,0,0,0]
print(get_flat_param_index("conv2", "weight", (0,0,0,0)))  # Output: 2432

# Index of fc1.weight[0, 0]
print(get_flat_param_index("fc1", "weight", (0, 0)))  # Output: 53696

In [ ]:
results4_loc = []

for i in range(0, 32):
    results4_loc.append(inj.run_seu_autodiagonal_normal(location_index=2400, bit_i=i, parameter_name="loc", num_samples=10))

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(range(0, 32), results4_loc, marker='o')
plt.title('SEU Impact on Loc Parameter')
plt.xlabel('Bit Position')
plt.ylabel('Change in Accuracy')
plt.grid()
plt.show()

In [ ]:
# plot results2_loc, results3_loc, results4_loc in a single plot
plt.figure(figsize=(10, 5))
plt.plot(range(0, 32), results_loc, marker='o', label='conv1 1st weight')
plt.plot(range(0, 32), results4_loc, marker='o', label='conv1 1st bias')
plt.plot(range(0, 32), results3_loc, marker='o', label='fc1 1st weight')
plt.plot(range(0, 32), results2_loc, marker='o', label='fc1 1st bias')


plt.title('SEU Impact on Loc Parameter')
plt.xlabel('Bit Position')
plt.ylabel('Change in Accuracy')
plt.grid()
plt.legend()
plt.show()

In [ ]:
#results
#results2_scale
#...
#...

In [ ]:
results_scale = []

for i in range(0, 32):
    results_scale.append(inj.run_seu_autodiagonal_normal(location_index=0, bit_i=i, parameter_name="scale", num_samples=10))

In [ ]:
#store the result in dataframe
import pandas as pd
results2_scale = []

for i in range(0, 32):
    results2_scale.append(inj.run_seu_autodiagonal_normal(location_index=217536, bit_i=i, parameter_name="scale", num_samples=10))

In [ ]:
#store the result in dataframe
import pandas as pd
results3_scale = []

for i in range(0, 32):
    results3_scale.append(inj.run_seu_autodiagonal_normal(location_index=53696, bit_i=i, parameter_name="scale", num_samples=10))

In [ ]:
#store the result in dataframe
import pandas as pd
results4_scale = []

for i in range(0, 32):
    results4_scale.append(inj.run_seu_autodiagonal_normal(location_index=2400, bit_i=i, parameter_name="scale", num_samples=10))

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(range(0, 32), results_scale, marker='o', label='conv1 1st weight')
plt.plot(range(0, 32), results4_scale, marker='o', label='conv1 1st bias')
plt.plot(range(0, 32), results3_scale, marker='o', label='fc1 1st weight')
plt.plot(range(0, 32), results2_scale, marker='o', label='fc1 1st bias')


plt.title('SEU Impact on Scale Parameter')
plt.xlabel('Bit Position')
plt.ylabel('Change in Accuracy')
plt.grid()
plt.legend()
plt.show()